In [1]:
#import all the dependences 
from langchain.chains import LLMChain , SequentialChain 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import json
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

In [ ]:
# Add Openai API and Base link 

API_KEY = api_key
BASE_URL = "https://api.sree.shop/v1"

In [ ]:
# Create a llm of model
llm = ChatOpenAI(
    openai_api_key = API_KEY,
    base_url = BASE_URL,
    model_name = 'gpt-4o',
    temperature = 0.7,
)

In [ ]:
# Create json format in which we need output 
RESPONSE_JSON = {
    "1": {
        "mcq" : "Choice the correct answer?",
        "options" : {
            'a' : 'choice here',
            'b' : 'choice here',
            'c' : 'choice here',
            'd' : 'choice here',        
        },
        "correct answer" : "correct answer here",
    },
    "2": {
        "mcq" : "Choice the correct answer?",
        "options" : {
            'a' : 'choice here',
            'b' : 'choice here',
            'c' : 'choice here',
            'd' : 'choice here',        
        },
        "correct answer" : "correct answer here",
    },
    "3": {
        "mcq" : "Choice the correct answer?",
        "options" : {
            'a' : 'choice here',
            'b' : 'choice here',
            'c' : 'choice here',
            'd' : 'choice here',        
        },
        "correct answer" : "correct answer here",
    },
}

In [ ]:
# Let's create a template of input
Template = """
Text: {text}
You are an Expert MCQ maker. We gave you a text above. Make {number} MCQs from the given text for {subject} to a student with the difficulty of {tone}. 
Make sure all the MCQs are not repeated and confirm from the text as well. 
Ensure your response follows this template: RESPONE_JSON. Ensure the MCQs count matches {number}.
{response_json}
"""

In [ ]:
student_prompt = PromptTemplate(
    input_variables = ['text' , 'number' ,'subject', 'tone', 'response_json'],
    template = Template,
)

In [ ]:
student_chain = LLMChain(
    llm = llm,
    prompt = student_prompt,
    output_key = 'quiz',
    verbose = True
)

In [ ]:
# Lets define second template for evaluating 1st response
Template2 = """
You are an expert writer and examiner of {subject} for students.
You need to evaluate the complexity of the MCQs and analyze them.
Use a maximum of 50 words to analyze the complexity. 
If the quiz is not appropriate for the cognitive and analytical abilities of the students, modify the questions to fit the required tone.
Quiz questions: {quiz}
"""

In [ ]:
teacher_prompt = PromptTemplate(
    input_variables = ['subject' , 'quiz'],
    template = Template2,
)

In [ ]:
teacher_chain = LLMChain(
    llm = llm, 
    prompt = teacher_prompt, 
    output_key = 'review',
    verbose = True,
)

In [ ]:
# Lets create a sequential chain for combing both respones 
final_chain = SequentialChain(
    chains = [student_chain , teacher_chain],
    input_variables = ['text' , 'number' ,'subject', 'tone', 'response_json'],
    output_variables = ['quiz' , 'review'],
    verbose = True,
)

In [ ]:
# Read a file 
file_path = r"C:\Users\SAJID HAMEED\Desktop\AI_WorkShop\AI_with_hugging_face\data.txt"
with open(file_path , 'r') as file:
    Text = file.read()


In [ ]:
# Serializing the python dictionary in to json format
json.dumps(RESPONSE_JSON)

In [ ]:
# lets input variables 
Number = 5 
Subject = 'machine learning'
Tone = 'Simple'

In [ ]:
response = final_chain(
    {
        'text' : Text,
        'subject': Subject,
        'number' : Number,
        'tone' : Tone,
        'response_json' : RESPONSE_JSON
    }
)

In [ ]:
quiz = response.get('quiz')
print(quiz)